<a href="https://colab.research.google.com/github/Madina069/Project_No.2_ML/blob/main/YouTube_and_Reddit_Comments_and_Replies_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Tutorial on comment extraction:
# https://praw.readthedocs.io/en/stable/tutorials/comments.html

In [1]:
API_key = "your API key"

In [2]:
# Function to get replies for a specific comment
def get_replies(youtube, parent_id, video_id):
    replies = []
    next_page_token = None

    while True:
        reply_request = youtube.comments().list(
            part="snippet",
            parentId=parent_id,
            textFormat="plainText",
            maxResults=100,
            pageToken=next_page_token
        )
        reply_response = reply_request.execute()

        for item in reply_response['items']:
            comment = item['snippet']
            replies.append({
                'Timestamp': comment['publishedAt'],
                'Username': comment['authorDisplayName'],
                'VideoID': video_id,
                'Comment': comment['textDisplay'],
                'Date': comment['updatedAt'] if 'updatedAt' in comment else comment['publishedAt']
            })

        next_page_token = reply_response.get('nextPageToken')
        if not next_page_token:
            break

    return replies

In [3]:
import googleapiclient.discovery
import pandas as pd
from google.colab import files, drive

# Build the YouTube client
youtube = googleapiclient.discovery.build('youtube', 'v3', developerKey=API_key)

# Function to get all comments (including replies) for a single video
def get_comments_for_video(youtube, video_id):
    all_comments = []
    next_page_token = None

    while True:
        comment_request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            pageToken=next_page_token,
            textFormat="plainText",
            maxResults=100
        )
        comment_response = comment_request.execute()

        for item in comment_response['items']:
            top_comment = item['snippet']['topLevelComment']['snippet']
            all_comments.append({
                'Timestamp': top_comment['publishedAt'],
                'Username': top_comment['authorDisplayName'],
                'VideoID': video_id,
                'Comment': top_comment['textDisplay'],
                'Date': top_comment['updatedAt'] if 'updatedAt' in top_comment else top_comment['publishedAt']
            })

            # Fetch replies if there are any
            if item['snippet']['totalReplyCount'] > 0:
                all_comments.extend(get_replies(youtube, item['snippet']['topLevelComment']['id'], video_id))

        next_page_token = comment_response.get('nextPageToken')
        if not next_page_token:
            break

    return all_comments

In [4]:
#Getting comments and replies

video_ids = ['9yFz5O7H5cY']

all_comments = []

for video_id in video_ids:
    video_comments = get_comments_for_video(youtube, video_id)
    all_comments.extend(video_comments)

comments_df = pd.DataFrame(all_comments)

In [5]:
comments_df.info()
print("\n---\n")
comments_df['VideoID'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354 entries, 0 to 353
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Timestamp  354 non-null    object
 1   Username   354 non-null    object
 2   VideoID    354 non-null    object
 3   Comment    354 non-null    object
 4   Date       354 non-null    object
dtypes: object(5)
memory usage: 14.0+ KB

---



VideoID
9yFz5O7H5cY    354
Name: count, dtype: int64

In [6]:
comments_df.head()

,Timestamp,Username,VideoID,Comment,Date
0,2024-06-11T22:01:37Z,@user-wf6hs6dn6f,9yFz5O7H5cY,I guess im moving to africa,2024-06-11T22:01:37Z
1,2024-05-26T13:38:36Z,@Iamnotxasin,9yFz5O7H5cY,I had this country,2024-05-26T13:38:36Z
2,2024-05-20T05:09:28Z,@XeroBritt,9yFz5O7H5cY,The horrendously ignorant and homophobic comme...,2024-05-20T05:09:28Z
3,2024-04-16T06:48:28Z,@user-hj6jd2ll1x,9yFz5O7H5cY,🇿🇲🇿🇲♥️ God bless Zambia,2024-04-16T06:48:28Z
4,2024-04-13T15:25:02Z,@veronicabrown9814,9yFz5O7H5cY,It must be common to their cities since they h...,2024-04-13T15:25:02Z


In [7]:
import requests
import requests.auth
!pip install -q praw
import praw
from praw.models import MoreComments

In [8]:
clientID = "jxfwSBK1AsCnoX5HxcnSIA"
clientSecret = "9zdEzBzvm_M4RZY2ZM9sQS9l0HIMoA"
username = "Emotional_Bowl_7415"
password = "datasciencetest"
userAgent = "GoogleColab:QuickAPITest:v0.1 (by /u/Emotional_Bowl_7415)"

client_auth = requests.auth.HTTPBasicAuth(clientID, clientSecret)
post_data = {"grant_type": "password", "username": username, "password": password}
headers = {"User-Agent": userAgent}
response = requests.post("https://www.reddit.com/api/v1/access_token", auth=client_auth, data=post_data, headers=headers)
response = response.json()

access_token = response['access_token']
token_type = response['token_type']

In [9]:
# create Reddit instance
reddit = praw.Reddit(
    client_id=clientID,
    client_secret=clientSecret,
    #optional
    username=username,
    password=password,
    user_agent=userAgent,
    check_for_async=False
)

url = "https://www.reddit.com/r/spotify/comments/e1ipwf/whats_up_with_the_lizzo_hate/"
submission = reddit.submission(url=url)

print("Total number of comments:", submission.num_comments, "\n")

submission.comments.replace_more(limit=None)

comments = []

for comment in submission.comments.list():
    comments.append(comment.body)

df = pd.DataFrame(comments)

df.info()
df.head()

Total number of comments: 63 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       38 non-null     object
dtypes: object(1)
memory usage: 432.0+ bytes


,0
0,"she has a nasally voice, her songs are adored ..."
1,I hated her music way before I knew she was ev...
2,Her Music sounds like a cat thrown up and a 12...
3,her music suckz\nAnd her lyrics are ass
4,She has nothing to sing about other than her a...


In [10]:
test_df = pd.concat([comments_df['Comment'], df[0]])
test_df.head()

0                          I guess im moving to africa
1                                   I had this country
2    The horrendously ignorant and homophobic comme...
3                              🇿🇲🇿🇲♥️ God bless Zambia
4    It must be common to their cities since they h...
dtype: object

In [11]:
test_df = pd.DataFrame(test_df)

In [12]:
test_df['Count'] = None
test_df.columns = ['Comment', 'Count']
test_df.head()

,Comment,Count
0,I guess im moving to africa,None
1,I had this country,None
2,The horrendously ignorant and homophobic comme...,None
3,🇿🇲🇿🇲♥️ God bless Zambia,None
4,It must be common to their cities since they h...,None


In [13]:
test_df[test_df['Comment'] == '[deleted]']

,Comment,Count
8,[deleted],None
31,[deleted],None
36,[deleted],None


In [14]:
test_df.drop([8,31,36], axis = 0, inplace = True)

In [15]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 386 entries, 0 to 37
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Comment  386 non-null    object
 1   Count    0 non-null      object
dtypes: object(2)
memory usage: 9.0+ KB


In [36]:
csv_file = 'test_data.csv'
test_df.to_csv(csv_file, index=False)

files.download(csv_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>